In [1]:
import numpy as np
import pandas as pd

from os import listdir
from os.path import basename, isfile, join

In [12]:
metadata = pd.read_csv(join('data', 'metadata.csv'))
metadata

,File Name,Class Name,Image Count
0,data/drill.npy,drill,136786
1,data/pool.npy,pool,133439
2,data/hexagon.npy,hexagon,142435
3,data/ice cream.npy,ice cream,123133
4,data/carrot.npy,carrot,132459
...,...,...,...
340,data/telephone.npy,telephone,127885
341,data/river.npy,river,133271
342,data/zigzag.npy,zigzag,120072
343,data/hat.npy,hat,222610


In [45]:
data = np.load('data/bat.npy', mmap_mode='r')

(118114, 784)

In [56]:
image = np.zeros(784)
image[:] = data[0][0]
data[0].shape[1]

784

In [74]:
data = []
data.append(np.random.rand(10,3))
data.append(np.random.rand(7,3))
data

[array([[0.39060065, 0.16652324, 0.88475379],
        [0.24948943, 0.94159072, 0.84889919],
        [0.42741121, 0.95396116, 0.24729562],
        [0.33231242, 0.58873905, 0.45828493],
        [0.48869594, 0.56932537, 0.25576275],
        [0.28818519, 0.46445471, 0.38454306],
        [0.89386452, 0.41782397, 0.15634985],
        [0.87942766, 0.17844109, 0.59297664],
        [0.16657688, 0.65783166, 0.5834018 ],
        [0.05482712, 0.94339731, 0.61702506]]),
 array([[0.69392878, 0.56849453, 0.44385202],
        [0.69012261, 0.53105313, 0.08851677],
        [0.83772191, 0.26538529, 0.9081723 ],
        [0.91756469, 0.16368993, 0.80635139],
        [0.33939133, 0.29861488, 0.88135504],
        [0.1909979 , 0.17917776, 0.10539897],
        [0.12918532, 0.69602445, 0.75020075]])]

In [75]:
counts = [10,7]
total = sum(counts)
labels = np.zeros(total, dtype=np.int64)
order = np.arange(total, dtype=np.int64)
idxs = np.zeros(total, dtype=np.int64)

start = 0
for idx,count in enumerate(counts):
    labels[start:start+count] = idx
    idxs[start:start+count] = np.arange(count)
    start += count

np.random.shuffle(order)
labels = labels[order]
idxs = idxs[order]

In [76]:
X = np.zeros((10,3))
labels = labels[:10]
idxs = idxs[:10]

for idx in range(10):
    X[idx,:] = data[labels[idx]][idxs[idx],:]
    print(X[idx,:], labels[idx], idxs[idx])

[0.33231242 0.58873905 0.45828493] 0 3
[0.69392878 0.56849453 0.44385202] 1 0
[0.48869594 0.56932537 0.25576275] 0 4
[0.42741121 0.95396116 0.24729562] 0 2
[0.24948943 0.94159072 0.84889919] 0 1
[0.1909979  0.17917776 0.10539897] 1 5
[0.87942766 0.17844109 0.59297664] 0 7
[0.69012261 0.53105313 0.08851677] 1 1
[0.28818519 0.46445471 0.38454306] 0 5
[0.39060065 0.16652324 0.88475379] 0 0


array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from tf.keras.utils import Sequence
from math import floor

class DataGenerator(Sequence):
    def __init__(self, metadata, batch_size):
        self.metadata = metadata
        self.batch_size = batch_size
        
        self.n_classes = self.metadata.shape[0]
        
        self.n_images = sum(self.metadata['Image Count'])
        
        self.indexes = np.zeros(self.n_images, dtype=np.int64)
        self.classes = np.zeros(self.n_images, dtype=np.int64)
        start = 0
        for idx,row in self.metadata.iterrows():
            count = row['Image Count']
            
            self.indexes[start:start+count] = np.arange(count)
            self.classes[start:start+count] = idx
            
            start += count
            
        order = np.arange(self.n_images, dtype=np.int64)
        np.random.shuffle(order)
        
        self.indexes = self.indexes[order]
        self.classes = self.classes[order]
        
        self.data = []
        for file in self.metadata['File Name']:
            self.data.append(np.load(file, mmap_mode='r'))
        
        self.size = self.data[0].shape[1]
            
        
    def __len__(self):
        return floor(self.n_images / self.batch_size)
    
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = (idx+1) * self.batch_size
        
        indexes = self.indexes[start:end]
        classes = self.classes[start:end]
        
        X = np.zeros((self.batch_size,self.size))
        y = classes
        
        for idx in range(self.batch_size):
            X[idx,:] = data[classes[idx]][indexes[idx],:]
        
        return X,y
        
        
        
        